# Scipy Sparse Matrices

**Sparse Matrices** are very nice in some situations.  

For example, in some machine learning tasks, especially those associated
with textual analysis, the data may be mostly zeros.  

Storing all these zeros is very inefficient.  

We can create and manipulate sparse matrices as follows:

In [1]:
import numpy as np

np.random.seed(42)

In [2]:
# Create a random array with a lot of zeros
X = np.random.random((10, 5))
print(X)

[[0.37454012 0.95071431 0.73199394 0.59865848 0.15601864]
 [0.15599452 0.05808361 0.86617615 0.60111501 0.70807258]
 [0.02058449 0.96990985 0.83244264 0.21233911 0.18182497]
 [0.18340451 0.30424224 0.52475643 0.43194502 0.29122914]
 [0.61185289 0.13949386 0.29214465 0.36636184 0.45606998]
 [0.78517596 0.19967378 0.51423444 0.59241457 0.04645041]
 [0.60754485 0.17052412 0.06505159 0.94888554 0.96563203]
 [0.80839735 0.30461377 0.09767211 0.68423303 0.44015249]
 [0.12203823 0.49517691 0.03438852 0.9093204  0.25877998]
 [0.66252228 0.31171108 0.52006802 0.54671028 0.18485446]]


In [3]:
X[X < 0.7] = 0
print(X)

[[0.         0.95071431 0.73199394 0.         0.        ]
 [0.         0.         0.86617615 0.         0.70807258]
 [0.         0.96990985 0.83244264 0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.78517596 0.         0.         0.         0.        ]
 [0.         0.         0.         0.94888554 0.96563203]
 [0.80839735 0.         0.         0.         0.        ]
 [0.         0.         0.         0.9093204  0.        ]
 [0.         0.         0.         0.         0.        ]]


In [4]:
from scipy import sparse

# turn X into a csr (Compressed-Sparse-Row) matrix
X_csr = sparse.csr_matrix(X)
print(X_csr)

  (0, 1)	0.9507143064099162
  (0, 2)	0.7319939418114051
  (1, 2)	0.8661761457749352
  (1, 4)	0.7080725777960455
  (2, 1)	0.9699098521619943
  (2, 2)	0.8324426408004217
  (5, 0)	0.7851759613930136
  (6, 3)	0.9488855372533332
  (6, 4)	0.9656320330745594
  (7, 0)	0.8083973481164611
  (8, 3)	0.9093204020787821


In [5]:
# convert the sparse matrix to a dense array
print(X_csr.toarray())

[[0.         0.95071431 0.73199394 0.         0.        ]
 [0.         0.         0.86617615 0.         0.70807258]
 [0.         0.96990985 0.83244264 0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.78517596 0.         0.         0.         0.        ]
 [0.         0.         0.         0.94888554 0.96563203]
 [0.80839735 0.         0.         0.         0.        ]
 [0.         0.         0.         0.9093204  0.        ]
 [0.         0.         0.         0.         0.        ]]


In [6]:
# Sparse matrices support linear algebra:
y = np.random.random(X_csr.shape[1])
z1 = X_csr.dot(y)
z2 = X.dot(y)
np.allclose(z1, z2)

True

* The CSR representation can be very efficient for computations, but it is not as good for adding elements.  

* For that, the **LIL** (List-In-List) representation is better:

In [7]:
# Create an empty LIL matrix and add some items
X_lil = sparse.lil_matrix((5, 5))

for i, j in np.random.randint(0, 5, (15, 2)):
    X_lil[i, j] = i + j

print(X_lil)
print(X_lil.toarray())

  (0, 2)	2.0
  (0, 3)	3.0
  (0, 4)	4.0
  (1, 0)	1.0
  (1, 1)	2.0
  (1, 4)	5.0
  (2, 0)	2.0
  (2, 2)	4.0
  (2, 4)	6.0
  (3, 0)	3.0
  (3, 3)	6.0
  (3, 4)	7.0
  (4, 0)	4.0
  (4, 4)	8.0
[[0. 0. 2. 3. 4.]
 [1. 2. 0. 0. 5.]
 [2. 0. 4. 0. 6.]
 [3. 0. 0. 6. 7.]
 [4. 0. 0. 0. 8.]]


* Often, once an LIL matrix is created, it is useful to convert it to a CSR format 
    * **Note**: many scikit-learn algorithms require CSR or CSC format

In [8]:
X_csr = X_lil.tocsr()
print(X_csr)

  (0, 2)	2.0
  (0, 3)	3.0
  (0, 4)	4.0
  (1, 0)	1.0
  (1, 1)	2.0
  (1, 4)	5.0
  (2, 0)	2.0
  (2, 2)	4.0
  (2, 4)	6.0
  (3, 0)	3.0
  (3, 3)	6.0
  (3, 4)	7.0
  (4, 0)	4.0
  (4, 4)	8.0


There are several other sparse formats that can be useful for various problems:

- `CSC` (compressed sparse column)
- `BSR` (block sparse row)
- `COO` (coordinate)
- `DIA` (diagonal)
- `DOK` (dictionary of keys)

## CSC - Compressed Sparse Column

**Advantages of the CSC format**

    * efficient arithmetic operations CSC + CSC, CSC * CSC, etc.
    * efficient column slicing
    * fast matrix vector products (CSR, BSR may be faster)

**Disadvantages of the CSC format**

    * slow row slicing operations (consider CSR)
    * changes to the sparsity structure are expensive (consider LIL or DOK)

### BSR - Block Sparse Row

The Block Compressed Row (`BSR`) format is very similar to the Compressed Sparse Row (`CSR`) format. 

BSR is appropriate for sparse matrices with *dense sub matrices* like the example below. 

Block matrices often arise in *vector-valued* finite element discretizations. 

In such cases, BSR is **considerably more efficient** than CSR and CSC for many sparse arithmetic operations.

In [9]:
from scipy.sparse import bsr_matrix

indptr = np.array([0, 2, 3, 6])
indices = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6]).repeat(4).reshape(6, 2, 2)
bsr_matrix((data,indices,indptr), shape=(6, 6)).toarray()

array([[1, 1, 0, 0, 2, 2],
       [1, 1, 0, 0, 2, 2],
       [0, 0, 0, 0, 3, 3],
       [0, 0, 0, 0, 3, 3],
       [4, 4, 5, 5, 6, 6],
       [4, 4, 5, 5, 6, 6]])

## COO - Coordinate Sparse Matrix

**Advantages of the CSC format**

    * facilitates fast conversion among sparse formats
    * permits duplicate entries (see example)
    * very fast conversion to and from CSR/CSC formats

**Disadvantages of the CSC format**

    * does not directly support arithmetic operations and slicing
    
** Intended Usage**

    * COO is a fast format for constructing sparse matrices
    * Once a matrix has been constructed, convert to CSR or CSC format for fast arithmetic and matrix vector
    operations
    * By default when converting to CSR or CSC format, duplicate (i,j) entries will be summed together. 
    This facilitates efficient construction of finite element matrices and the like.


## DOK - Dictionary of Keys

Sparse matrices can be used in arithmetic operations: they support addition, subtraction, multiplication, division, and matrix power.

Allows for efficient O(1) access of individual elements. Duplicates are not allowed. Can be efficiently converted to a coo_matrix once constructed.

In [10]:
from scipy.sparse import dok_matrix
S = dok_matrix((5, 5), dtype=np.float32)
for i in range(5):
    for j in range(i, 5):
        S[i,j] = i+j
        
S.toarray()

array([[0., 1., 2., 3., 4.],
       [0., 2., 3., 4., 5.],
       [0., 0., 4., 5., 6.],
       [0., 0., 0., 6., 7.],
       [0., 0., 0., 0., 8.]], dtype=float32)

The ``scipy.sparse`` submodule also has a lot of functions for sparse matrices
including linear algebra, sparse solvers, graph algorithms, and much more.